# Multiclass classification
In this notebook we will make a multiclass classification, the starting paper is as follows: [A Deep Learning Approach for Viral DNA Sequence Classification using Genetic Algorithm](https://www.researchgate.net/publication/363276607_A_Deep_Learning_Approach_for_Viral_DNA_Sequence_Classification_using_Genetic_Algorithm)

We will propose different models for the task on a different dataset


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Definition of common import and functions


In [3]:
from json import encoder
import json
import pandas as pd

import keras

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from tqdm import tqdm
from matplotlib import pyplot as plt

df_path = '/content/drive/MyDrive/Dataset/MulticlassDatasets/'
epochs = 20

## Models

In [4]:
def get_model(dimension, num_classes):
    inputs = keras.layers.Input(shape=dimension)
    x = keras.layers.Masking(-1)(inputs)
    x = keras.layers.LSTM(1024)(x)
    x = keras.layers.Dropout(0.2)(x)
    x = keras.layers.Dense(256)(x)
    x = keras.layers.Dropout(0.2)(x)
    x = keras.layers.Dense(num_classes)(x)
    x = keras.layers.Activation('softmax')(x)
    model = keras.models.Model(inputs=inputs, outputs=x)
    return model


### Unbalanced data set loading

In [5]:
df = None
with open(df_path+'UnBalancedDataset.jsonl') as f:
  for index, line in tqdm(enumerate(f)):
      data = json.loads(line)
      if df is None:
          df = pd.DataFrame(data=data, index=[0])
      else:
          df.loc[len(df)] = data
df.head()

1949it [00:07, 268.98it/s]


,Accession,Release Date,Species,Genus,Family,Molecule Type,Length,Sequence Type,Host,Collection Date,Sequence
0,NC_077680.1,2023-05-06T00:00:00Z,Tomato mottle leaf curl virus,Begomovirus,Geminiviridae,ssDNA(+/-),2630.0,RefSeq,Solanum lycopersicum,2008,ACCGGATGGCCGCGCGGGTTTTTTTGACCCGCTCCGTGATGTATTT...
1,NC_077711.1,2023-05-06T00:00:00Z,Sida micrantha mosaic virus,Begomovirus,Geminiviridae,ssDNA(+/-),2659.0,RefSeq,nan,nan,ACCGGATGGCCGCGCGATTTTCCCCCCCCTCACGTGGCGCTCTGGT...
2,NC_077712.1,2023-05-06T00:00:00Z,Sida micrantha mosaic virus,Begomovirus,Geminiviridae,ssDNA(+/-),2629.0,RefSeq,nan,nan,ACCGGATGGCCGCGCGATTTTCCCCCCAAAACGTGGCGCTCTGGTG...
3,NC_077719.1,2023-05-06T00:00:00Z,Cotton yellow mosaic virus,Begomovirus,Geminiviridae,ssDNA(+/-),2766.0,RefSeq,Gossypium raimondii,2014,ACCGGATGGCCGCGCGCCCGCTTTATGTGGTCCCCCCTTGTGGTCC...
4,NC_077720.1,2023-05-06T00:00:00Z,Cotton yellow mosaic virus,Begomovirus,Geminiviridae,ssDNA(+/-),2716.0,RefSeq,Gossypium raimondii,2014,ACCGGATGGCCGCGCGCCCCCTTTTATGTGGCCCACACACAGGATA...


In [6]:
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,Accession,Release Date,Species,Genus,Family,Molecule Type,Length,Sequence Type,Host,Collection Date,Sequence
0,NC_003737.1,2001-12-14T00:00:00Z,Rice black streaked dwarf virus,Fijivirus,Spinareoviridae,dsRNA,2645.0,RefSeq,Oryza sativa,nan,AAGTTTTTTGAGTCTGAGATACTGTTACCCATTGCGACCCTGAGAA...
1,NC_038401.1,2018-08-24T00:00:00Z,Cyclovirus nietoperz,Cyclovirus,Circoviridae,ssDNA(+/-),1809.0,RefSeq,Plecotus auritus,2013-06,ATGGCAAACAGTACAGTAAGGAGGTTCTGTTTCACGTGGAACAACT...
2,NC_007220.1,2005-08-02T00:00:00Z,Circovirus duck,Circovirus,Circoviridae,ssDNA(+/-),1991.0,RefSeq,Anas platyrhynchos,nan,ACCGGCGCTTGTACTCCGTACTCCGGGCCAAGGGGATAGCAACTGC...
3,NC_055355.1,2021-06-01T00:00:00Z,Zaliv Terpeniya uukuvirus,Uukuvirus,Phenuiviridae,RNA,1721.0,RefSeq,Ixodoidea,nan,ACACAAAGACCTCCAACTAAGCTATCGATTCATCATGGCTATTCCG...
4,NC_025492.1,2014-11-14T00:00:00Z,Fako virus,Dinovernavirus,Spinareoviridae,dsRNA,1778.0,RefSeq,Culicidae,2010,AGTTTAAAACCCTTCTGATTAGCAATGTTTGCAATACCTTTTACAG...


In [7]:
labels = df['Family']
X = df['Sequence']
X

0       AAGTTTTTTGAGTCTGAGATACTGTTACCCATTGCGACCCTGAGAA...
1       ATGGCAAACAGTACAGTAAGGAGGTTCTGTTTCACGTGGAACAACT...
2       ACCGGCGCTTGTACTCCGTACTCCGGGCCAAGGGGATAGCAACTGC...
3       ACACAAAGACCTCCAACTAAGCTATCGATTCATCATGGCTATTCCG...
4       AGTTTAAAACCCTTCTGATTAGCAATGTTTGCAATACCTTTTACAG...
                              ...                        
1944    CAGTATTACCCGAGACATCGGCACACTTCGGCACAATCGAGCGGCG...
1945    ACCGCTCGGCCCGAAAAAATCGCCGTACGATGTGACGTTTTTGAGC...
1946    ACACAAAGTCCAGGGCATTTTACAATATTCTATTCAATCCACTATC...
1947    ACCGGATGGCCGCCCGCAACCACGCCCCGACTCGCGCGTGTCTTAC...
1948    ACACAGAGACGGCTATACATTAAAGTAGAGGTAAACCGTAATCCAC...
Name: Sequence, Length: 1949, dtype: object

I see the maximum length of the sequence so that I can make the dataset containing sequence and target

In [8]:
max_len = []
for el in X:
    max_len.append(len(el))
max_len = max(max_len)
max_len

3999

In [9]:
cols =[]

for col in range(max_len):
    cols.append('nucleotide_' + str(col))


In [10]:
def encoder_sequence(seq):
  if seq == 'A':
    return 1
  elif seq == 'C':
    return 2
  elif seq == 'G':
    return 3
  elif seq == 'T':
    return 4
  else:
    return -1

def decode_sequence(seq):
  if seq == 1:
    return 'A'
  elif seq == 2:
    return 'C'
  elif seq == 3:
    return 'G'
  elif seq == 4:
    return 'T'
  else:
    return None

In [12]:
sequence_df = pd.DataFrame(columns = cols)

for row in tqdm(X):
    sequence = {}
    row = list(row)
    for index, i in enumerate(cols):
        if index < len(row):
            sequence[i] = encoder_sequence(row[index])
        else:
          sequence[i] = None

    sequence_df.loc[len(sequence_df)] = sequence

sequence_df.head()

100%|██████████| 1949/1949 [31:22<00:00,  1.04it/s]


,nucleotide_0,nucleotide_1,nucleotide_2,nucleotide_3,nucleotide_4,nucleotide_5,nucleotide_6,nucleotide_7,nucleotide_8,nucleotide_9,...,nucleotide_3989,nucleotide_3990,nucleotide_3991,nucleotide_3992,nucleotide_3993,nucleotide_3994,nucleotide_3995,nucleotide_3996,nucleotide_3997,nucleotide_3998
0,1,1,3,4,4,4,4,4,4,3,...,None,None,None,None,None,None,None,None,None,None
1,1,4,3,3,2,1,1,1,2,1,...,None,None,None,None,None,None,None,None,None,None
2,1,2,2,3,3,2,3,2,4,4,...,None,None,None,None,None,None,None,None,None,None
3,1,2,1,2,1,1,1,3,1,2,...,None,None,None,None,None,None,None,None,None,None
4,1,3,4,4,4,1,1,1,1,2,...,None,None,None,None,None,None,None,None,None,None


## Convert to numerical and fit the model

In [48]:
encoder_y = OneHotEncoder()
X = sequence_df.to_numpy().astype(np.float32)
y = labels.to_numpy()
encoder_y.fit(y.reshape(-1,1))

X.shape, y.shape

((1949, 3999), (1949,))

In [49]:
dimension = (X.shape[1], 1)
dimension

(3999, 1)

In [50]:
n_class = len(labels.unique())
# param for the model
batch_size = 32
n_class

4

In [51]:
model = get_model(dimension, n_class)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 3999, 1)]         0         
                                                                 
 masking_5 (Masking)         (None, 3999, 1)           0         
                                                                 
 lstm_5 (LSTM)               (None, 1024)              4202496   
                                                                 
 dropout_10 (Dropout)        (None, 1024)              0         
                                                                 
 dense_10 (Dense)            (None, 256)               262400    
                                                                 
 dropout_11 (Dropout)        (None, 256)               0         
                                                                 
 dense_11 (Dense)            (None, 4)                 1028

In [ ]:
import numpy as np
# Stratified K-Fold cross validation
skf = StratifiedKFold(n_splits=10, shuffle=True)

n_fold = 0
best_accuracy = 0
best_model = None
best_pred = None
gt_pred = None

for train_index, test_index in skf.split(X, y):
    # Get the train data and split it in train and val
    X_f, y_f = X[train_index],y[train_index]
    X_train, X_val, y_train, y_val = train_test_split(X_f, y_f, test_size=0.2, random_state=42)
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)


    y_train = encoder_y.transform(y_train.reshape(-1,1)).toarray().astype(np.float32)
    y_val = encoder_y.transform(y_val.reshape(-1,1)).toarray().astype(np.float32)


    # fit the model
    history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val),batch_size=batch_size, verbose=1)

    y_pred = model.predict(X[test_index].reshape(X[test_index].shape[0], X[test_index].shape[1], 1))
    y_pred = encoder_y.inverse_transform(y_pred)

    acc = accuracy_score(y[test_index], y_pred)
    if acc > best_accuracy:
        best_accuracy = acc
        best_model = model
        best_pred = y_pred
        gt_pred = y[test_index]

    # plot the fit metrics
    fig, ax = plt.subplots(figsize=(1, 2))
    # summarize history for accuracy
    ax[0].plot(history.history['acc'])
    ax[0].plot(history.history['val_acc'])
    ax[0].legend(['train', 'val'])
    ax[1].plot(history.history['loss'])
    ax[1].plot(history.history['val_loss'])
    ax[1].legend(['train', 'val'])
    fig.title(f'Fold number {n_fold}')
    n_fold += 1

Epoch 1/20
23/44 [==============>...............] - ETA: 1:14:53 - loss: nan - accuracy: 0.1495

In [ ]:
print(classification_report(y_true=gt_pred, y_pred=best_pred))

In [ ]:
print(confusion_matrix(y_true=gt_pred, y_pred=best_pred))

## Explainability of the model